In [36]:
import numpy as np
from scipy import optimize
from sympy import nsolve
import matplotlib.pyplot as plt



# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from Funcs import *
from Model import *
model = production_economy()
model.setup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1
We first find the market clearing conditions, i.e. the value of $l^\star$, for the different prices defined as $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`.

In [37]:
# Define the linspaces
p1_value = np.linspace(0.1, 2.0, 10)
p2_value = np.linspace(0.1, 2.0, 10)

# Empty array to store results
results = []

for p1 in p1_value:
    for p2 in p2_value:
        if model.market_clearing(p1, p2) != None:
            labor, good1, good2 = model.market_clearing(p1, p2)
            results.append([p1, p2, labor, good1, good2])

            # Print the results
            for result in results:
                p1, p2, labor, good1, good2 = result
                # print(f"p1: {p1:.2f}, p2: {p2:.2f}, Labor MC: {labor:.4f}, Good1 MC: {good1:.4f}, Good2 MC: {good2:.4f}")

# Print number of markets clearing
print(f"Number of markets not clearing: {len(p1_value)*len(p2_value) - len(results)}")
print(f"Number of markets clearing: {len(results)}")


Prices of good 1: 0.1
Prices of good 2: 0.1
Excess demand: [0.095      2.96000834 6.97335279]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.3111111111111111
Excess demand: [0.17885802 3.0036313  2.13466792]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.5222222222222223
Excess demand: [0.2404321  3.09299709 1.14320674]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.7333333333333333
Excess demand: [0.27972222 3.22995033 0.67695389]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.9444444444444444
Excess demand: [0.2967284  3.41709192 0.38435343]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.1555555555555557
Excess demand: [0.29145062 3.65757128 0.17086642]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.3666666666666667
Excess demand: [2.63888889e-01 3.95483660e+00 4.19256647e-04]
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.577777777777778
Excess demand: [ 0.21

# Question 2
We now want to find the equilibrium prices p1 and p2 that ensures market clearing. 
We will use Walras' law such that by having 2 markets lcear, the clearing of market 3 is ensured.

In [38]:
initial_guess = [model.par.p1, model.par.p2]

equilibrium_prices = optimize.fsolve(Walras_law, initial_guess, args=(model.par.tau, model.par.T, model.par))

# Print the results
p1_equilibrium, p2_equilibrium = equilibrium_prices
model.par.p1 = p1_equilibrium
model.par.p2 = p2_equilibrium
print(f"Equilibrium prices: p1 = {p1_equilibrium:.4f}, p2 = {p2_equilibrium:.4f}")

# Calculate the equilibrium values
laborD_eq = labor_demand(model.par, p1_equilibrium) + labor_demand(model.par, p2_equilibrium)
laborS_eq = labor_supply(model.par, p1_equilibrium, p2_equilibrium, model.par.tau, model.par.T)
good1_eq, good2_eq = demand(model.par, p1_equilibrium, p2_equilibrium, model.par.tau, model.par.T)
y1_eq = production(model.par, p1_equilibrium)
y2_eq = production(model.par, p2_equilibrium)

# Print the results
print(f"Equilibrium values: Labor demand = {laborD_eq:.4f}, Labor supply = {laborS_eq:.4f}, Good1 = {good1_eq:.4f}, Good2 = {good2_eq:.4f}, Y1 = {y1_eq:.4f}, Y2 = {y2_eq:.4f}")

Equilibrium prices: p1 = 0.9759, p2 = 1.4908
Equilibrium values: Labor demand = 0.7937, Labor supply = 0.7937, Good1 = 0.4880, Good2 = 0.7454, Y1 = 0.4880, Y2 = 0.7454


We see that both goods market clears, and thus according to Walras law we know that the market for labor supply and demand must also clear.  
The solutions are thus:  
$c_1=y_1=0.4880$  
$c_2=y_2=0.7454$  
$\ell^\star=0.7937$

# Question 3
The government now cares about the social welfare function:
$$SWF=U-\kappa y_2^\star$$
We now want to find the value of tax, $\tau$, and lump sum transfers, $T$, that maximizes this function.  
We are told that the governmen balances its budget such that $T=\tau c_2^\star$.

In [39]:
# Defining an objective function to optimize where the government chooses the optimal tax rate and transfers to maximize the wocial welfare function and the prices clear the markets

def objective_function(x, par):

    tau, p1, p2, T = x

    SW = utility_SWF(par, tau, T, p1, p2) - par.kappa*production(par, p2)

    return -SW

# Initial guess
initial_guess = [-0.1, 1.0, 1.0, 1.0]

# Constraints - Prices have to clear the markets
constraints = ({'type': 'eq', 'fun': lambda x: labor_supply(model.par, x[1], x[2], x[0], x[3]) - labor_demand(model.par, x[1]) - labor_demand(model.par, x[2])},
               {'type': 'eq', 'fun': lambda x: production(model.par, x[2]) - demand(model.par, x[1], x[2], x[0], x[3])[1]},
               {'type': 'eq', 'fun': lambda x: x[3] - x[0] * production(model.par, x[2])})

# Optimize the objective function
optimal_tax = optimize.minimize(objective_function, initial_guess, args=(model.par,), constraints=constraints, method='SLSQP', tol=1e-10)
tau_optimal = optimal_tax.x[0]
p1 = optimal_tax.x[1]
p2 = optimal_tax.x[2]

model.par.tau = tau_optimal
T_optimal = tau_optimal * production(model.par, p2)
model.par.T = T_optimal

# Print the results
print(f"Optimal tax rate: {tau_optimal:.4f}")
print(f"Optimal transfer: {model.par.T:.4f}")
print(f"Optimal prices: p1 = {p1:.4f}, p2 = {p2:.4f}")

gov_budget = T_optimal - tau_optimal * production(model.par, p2)
print(f"Government budget: {gov_budget:.4f}")

laborD_eq = labor_demand(model.par, p1) + labor_demand(model.par, p2)
laborS_eq = labor_supply(model.par, p1, p2, model.par.tau, model.par.T)
good1_eq, good2_eq = demand(model.par, p1, p2, model.par.tau, model.par.T)
y1_eq = production(model.par, p1)
y2_eq = production(model.par, p2)

# Print the results
print(f"Equilibrium values: Labor demand = {laborD_eq:.4f}, Labor supply = {laborS_eq:.4f}, Good1 = {good1_eq:.4f}, Good2 = {good2_eq:.4f}, Y1 = {y1_eq:.4f}, Y2 = {y2_eq:.4f}")


Optimal tax rate: 0.3053
Optimal transfer: 0.2157
Optimal prices: p1 = 1.0201, p2 = 1.4130
Government budget: 0.0000
Equilibrium values: Labor demand = 0.7593, Labor supply = 0.7593, Good1 = 0.5101, Good2 = 0.7065, Y1 = 0.5101, Y2 = 0.7065


We see that the optimal tax is $0.3$ while the optimal transfer is $0.2$. This makes the price for good 2 drop to $1.41$ as the demand decreases and the consumers substitutes to good 1.  
Again, we choose to clear the labor market and the market for good 2, as the market for good 1 will then also clear according to Walras' law.  
Likely bacuse of the transfer the consumers now work slightly less, and as expected they consume more of good 1 and less of good 2, compared to question 2.